In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv 
import os 
from sql_functions import encrypt
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)


In [2]:
load_dotenv()

Python-dotenv could not parse statement starting at line 9


True

In [6]:
# Define the API endpoint, headers and used variables
url = 'https://www.wixapis.com/stores/v2/inventoryItems/query'
rows_offset = 0
counter = [1]
data = pd.DataFrame()
inventory_df = pd.DataFrame()
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json, text/plain, */*',
    'Authorization': os.getenv("wix_api_key"),
    'wix-site-id': os.getenv("statistik_side_id")  
}

for i in counter:
    # Define the query payload as a dictionary
    query_payload = {
        "query": {
            "paging": {
                "offset": rows_offset
            }
        }
    }

    # Make an HTTP POST request
    response = requests.post(url, json=query_payload, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        data = data.get("inventoryItems")

        # Normalize and concatenate the wanted data
        temporary_df = pd.json_normalize(data, sep="_") 
        inventory_df = pd.concat([temporary_df,inventory_df])

        # Set counter and offset
        rows_offset += 100
        counter.append(1)

    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        print("Or end of request. Check wix_orders's content")
        break
    


Request failed with status code 400: {"message":"Illegal query offset and limit values. Offset + limit must be less than or equal to 10000. Requested offset: 10000, requested limit: 100","details":{"validationError":{"fieldViolations":[{"field":"Paging","description":"Illegal query offset and limit values. Offset + limit must be less than or equal to 10000. Requested offset: 10000, requested limit: 100","violatedRule":"OTHER","ruleName":"IllegalQueryOffsetAndLimitSum"}]}}}
Or end of request. Check wix_orders's content


In [ ]:
inventory_df

In [12]:
testerei = inventory_df["variants"].explode()
testerei = pd.json_normalize(testerei, sep="_")
testerei

,variantId,inStock,availableForPreorder,quantity
0,00000000-0000-0000-0000-000000000000,True,False,NaN
1,00000000-0000-0000-0000-000000000000,True,False,NaN
2,00000000-0000-0000-0000-000000000000,True,False,NaN
3,00000000-0000-0000-0000-000000000000,True,False,NaN
4,00000000-0000-0000-0000-000000000000,True,False,NaN
...,...,...,...,...
167,00000000-0000-0000-0000-000000000000,True,False,NaN
168,00000000-0000-0000-0000-000000000000,True,False,NaN
169,00000000-0000-0000-0000-000000000000,True,False,NaN
170,00000000-0000-0000-0000-000000000000,True,False,NaN
